In [1]:
import polars as pl

tracking_df = pl.read_parquet("../data/split_prepped_data/*_features.parquet")
play_df = pl.read_csv("../data/bdb_2024/plays.csv", null_values=["", "NA", "na", "nan", "NaN", "NAN"]).with_columns(
    distanceToGoal=(
        pl.when(pl.col("possessionTeam") == pl.col("yardlineSide"))
        .then(100 - pl.col("yardlineNumber"))
        .otherwise(pl.col("yardlineNumber"))
    )
)

play_df.sample(3)

gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2,distanceToGoal
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,i64,i64,i64,str,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,str,str,i64,str,i64
2022102305,172,43334,"""Derrick Henry""","""(12:30) R.Tannehill pass short…",1,2,6,"""TEN""","""IND""","""TEN""",36,"""12:30""",0,0,"""C""",-3,null,-3,-3,"""N""",74,"""SINGLEBACK""",7,0.708235,0.607546,0.392454,-0.039723,0.039723,1.807466,-1.475571,null,null,null,null,64
2022100600,3282,46367,"""Phillip Lindsay""","""(13:28) P.Lindsay right tackle…",4,1,10,"""IND""","""DEN""","""DEN""",49,"""13:28""",9,6,null,null,-10,7,-10,"""Y""",59,"""SINGLEBACK""",6,null,0.579682,0.420318,0.047958,-0.047958,2.821824,-1.082868,"""Offensive Holding""",null,52557,null,49
2022102400,3006,47856,"""David Montgomery""","""(13:03) D.Montgomery left tack…",4,1,5,"""CHI""","""NE""","""NE""",5,"""13:03""",14,26,null,null,null,4,4,"""N""",105,"""I_FORM""",8,0.357763,0.041665,0.958335,-0.005961,0.005961,5.813514,0.350489,null,null,null,null,5


In [2]:
results_df = (
    pl.concat(
        [
            pl.read_parquet("../models/best_models/zoo/best_model_results.parquet"),
            pl.read_parquet("../models/best_models/transformer/best_model_results.parquet"),
        ],
        how="diagonal",
    ).join(
        tracking_df.filter(pl.col("is_ball_carrier") == 1)
        .select(["x", "y", "gameId", "playId", "frameId", "mirrored", "event"])
        .rename({"x": "ball_carrier_x", "y": "ball_carrier_y"}),
        on=["gameId", "playId", "frameId", "mirrored"],
        how="inner",
    )
    .with_columns(
        frames_before_tackle=(pl.col("tackle_frameId") - pl.col("frameId")),
    )
)
print(len(results_df))
results_df.sample(10)

2118428


gameId,playId,mirrored,frameId,ballCarrierNflId,ballCarrierName,tackle_frameId,tackle_event,tackle_x,tackle_y,playResult,anchor_x,anchor_y,tackle_x_rel,tackle_y_rel,dataset_split,tackle_x_rel_pred,tackle_y_rel_pred,tackle_x_pred,tackle_y_pred,params,model_dim,num_layers,model_type,batch_size,dropout,learning_rate,num_heads,dim_feedforward,ball_carrier_x,ball_carrier_y,event,frames_before_tackle
i64,i64,bool,i64,i64,str,i64,str,f64,f64,i64,f64,f64,f64,f64,str,f32,f32,f64,f64,i32,i32,i32,str,i32,f64,f64,i32,i32,f64,f64,str,i64
2022100903,2995,false,15,41246,"""Brandin Cooks""",34,"""tackle""",43.24,38.93,7,34.19,47.73,9.05,-8.8,"""train""",9.03,-5.81,43.22,41.92,6638222,512,2,"""transformer""",256,0.3,0.0001,16,2048,38.07,45.21,null,19
2022102304,353,true,56,44849,"""Zay Jones""",63,"""out_of_bounds""",66.4,52.85,14,51.55,17.02,14.85,35.83,"""train""",17.219999,20.57,68.77,37.59,72502,128,2,"""zoo""",256,0.3,0.0001,null,null,62.98,48.63,"""first_contact""",7
2022092200,3007,true,6,46104,"""Nick Chubb""",61,"""tackle""",34.56,17.55,5,21.45,23.75,13.11,-6.2,"""train""",11.28,4.78,32.73,28.53,72502,128,2,"""zoo""",256,0.3,0.0001,null,null,21.45,23.75,"""ball_snap""",55
2022091803,3494,false,42,53703,"""Deon Jackson""",39,"""tackle""",31.88,17.55,-4,28.48,23.33,3.4,-5.78,"""train""",4.21,-6.15,32.69,17.18,6638222,512,2,"""transformer""",256,0.3,0.0001,16,2048,32.11,17.27,null,-3
2022101700,3694,true,39,40129,"""Latavius Murray""",54,"""tackle""",52.75,31.82,4,42.91,25.69,9.84,6.13,"""train""",9.27,4.16,52.18,29.85,6638222,512,2,"""transformer""",256,0.3,0.0001,16,2048,52.26,29.85,null,15
2022101603,3664,true,35,52733,"""James Robinson""",46,"""tackle""",75.59,40.17,6,62.64,29.87,12.95,10.3,"""train""",13.18,8.21,75.82,38.08,6638222,512,2,"""transformer""",256,0.3,0.0001,16,2048,72.56,35.33,"""first_contact""",11
2022102309,3124,false,82,44856,"""Gerald Everett""",84,"""tackle""",57.32,49.52,24,45.08,24.95,12.24,24.57,"""train""",15.89,22.92,60.97,47.87,72502,128,2,"""zoo""",256,0.3,0.0001,null,null,57.41,49.35,null,2
2022092900,2917,true,51,44860,"""Joe Mixon""",75,"""tackle""",108.7,34.16,2,102.32,32.6,6.38,1.56,"""train""",6.24,-0.32,108.56,32.28,6638222,512,2,"""transformer""",256,0.3,0.0001,16,2048,109.16,31.7,null,24
2022110300,3506,false,68,52461,"""Jalen Hurts""",78,"""fumble""",44.51,7.7,14,24.71,23.58,19.8,-15.88,"""train""",19.299999,-18.99,44.01,4.59,72502,128,2,"""zoo""",256,0.3,0.0001,null,null,41.74,12.3,null,10


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


def animate_play(
    tracking_df: pl.DataFrame,
    play_df: pl.DataFrame,
    results_df: pl.DataFrame,
    gameId: int,
    playId: int,
    mirrored: bool = False,
):
    """
    Animate a play from the tracking data and the results of the models.
    """
    mvmt_df = (
        tracking_df.filter(
            (pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)
        )
        .to_pandas()
        .round(2)
    )
    play_df = play_df.filter((pl.col("gameId") == gameId) & (pl.col("playId") == playId)).to_pandas()
    model_results_df = results_df.filter(
        (pl.col("gameId") == gameId) & (pl.col("playId") == playId) & (pl.col("mirrored") == mirrored)
    ).to_pandas()
    assert len(mvmt_df) > 0
    assert len(play_df) > 0
    assert len(model_results_df) > 0

    mvmt_df["side"] = mvmt_df["side"].replace({1: "OFF", -1: "DEF"})
    # display(mvmt_df.sample(3), play_df, model_results_df)

    # get some info
    distToGoal = play_df["distanceToGoal"].values[0]
    down = play_df["down"].values[0]
    yards_to_go = play_df["yardsToGo"].values[0]
    play_description = play_df["playDescription"].values[0]
    ballCarrierName = model_results_df["ballCarrierName"].values[0]
    off_club = mvmt_df.loc[mvmt_df["side"] == "OFF", "club"].values[0]
    def_club = mvmt_df.loc[mvmt_df["side"] == "DEF", "club"].values[0]
    dataset_split = model_results_df["dataset_split"].values[0]

    mvmt_y_min = mvmt_df["x"].min()
    mvmt_y_max = mvmt_df["x"].max()

    tkl_x, tkl_y = model_results_df[["tackle_x", "tackle_y"]].values[0]
    # Get prediction data for animation
    tkl_mvmt_df = model_results_df.sort_values("frameId")[["model_type", "frameId", "tackle_x_pred", "tackle_y_pred"]]
    tkl_mvmt_df = tkl_mvmt_df.rename(columns={"tackle_x_pred": "x", "tackle_y_pred": "y"})
    tkl_mvmt_df["displayName"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["nflId"] = tkl_mvmt_df["model_type"].map({"zoo": -10, "transformer": -20})
    tkl_mvmt_df["club"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["side"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["symbol"] = tkl_mvmt_df["model_type"]
    tkl_mvmt_df["size"] = 1
    tkl_mvmt_df["jerseyNumber"] = ""
    tkl_mvmt_df["is_ball_carrier"] = -1

    # set some things
    # colors_df = pl.read_csv("../data/team_colors.csv")
    # club_color_map = dict(colors_df.select(["club", "secondaryCol"]).rows())
    mvmt_df["size"] = 2
    mvmt_df["text_color"] = "black"

    # Different symbols for different positions
    mvmt_df["symbol"] = "player"
    mvmt_df.loc[mvmt_df["is_ball_carrier"] == 1, "symbol"] = "ball_carrier"
    mvmt_df.loc[mvmt_df["is_ball_carrier"] == 1, "size"] = 3
    # mvnt.loc[mvnt["side"] == "BALL", "symbol"] = "diamond_0"
    symbol_map = {"player": "circle", "ball_carrier": "hexagon", "zoo": "x", "transformer": "x"}

    # operations to do on both tackle and mvmt data
    mvmt_df = pd.concat([mvmt_df, tkl_mvmt_df]).sort_values(["frameId", "nflId"])
    mvmt_df["frameId_grp"] = mvmt_df["frameId"]
    mvmt_df = mvmt_df.groupby("frameId_grp").bfill()

    # display(mvmt_df.head(5))
    mvmt_df["frameId_event"] = mvmt_df["frameId"].astype(str) + mvmt_df["event"].fillna("").astype(str).apply(
        lambda x: f"_{x}" if len(x) > 0 else x
    )

    # Data to display on hover
    hover_data = {
        "displayName": True,
        "club": False,
        "side": False,
        "jerseyNumber": False,
        "is_ball_carrier": True,
        "symbol": False,
        "frameId": False,
        "x": True,
        "y": True,
        "vx": True,
        "vy": True,
        "size": False,
    }

    X_LEFT = 0
    X_MIDDLE = 160 / 6.0
    X_RIGHT = 160 / 3.0
    Y_MIN = 0
    Y_MAX = 120
    Y_MIDDLE = 60
    fig = px.scatter(
        data_frame=mvmt_df,
        x="y",
        y="x",
        animation_frame="frameId_event",
        animation_group="nflId",
        hover_name="displayName",
        hover_data=hover_data,
        text="jerseyNumber",
        width=1000,
        height=900,
        # range_x=[-160 / 6.0, 160 / 6.0],
        range_x=[X_LEFT - 2, X_RIGHT + 2],
        size="size",
        size_max=15,
        color="side",
        color_discrete_map={"OFF": "#39FF14", "DEF": "#FF69B4", "zoo": "blue", "transformer": "gold"},
        opacity=0.9,
        symbol="symbol",
        symbol_map=symbol_map,
    )

    # Add marker for tackle location
    fig.add_trace(
        go.Scatter(
            x=[tkl_y],
            y=[tkl_x],
            mode="markers",
            marker=dict(color="green", size=15, symbol="x"),
            hoverinfo="none",
            showlegend=False,
            opacity=0.8,
        )
    )

    # Add line of scrimmage
    los = Y_MAX - 10 - distToGoal
    fig.add_shape(
        type="line",
        x0=X_LEFT,
        y0=los,
        x1=X_RIGHT,
        y1=los,
        line=dict(color="rgba(137, 207, 240, 0.2)", width=3, dash="dash"),
    )
    # Add yards to go line
    fig.add_shape(
        type="line",
        x0=X_LEFT,
        y0=los + yards_to_go,
        x1=X_RIGHT,
        y1=los + yards_to_go,
        line=dict(color="rgba(255, 255, 0, 0.2)", width=3, dash="dash"),
    )
    # Add border to the field
    fig.add_shape(
        type="rect", x0=X_LEFT, y0=Y_MIN, x1=X_RIGHT, y1=Y_MAX, line=dict(color="rgba(255, 255, 255, 0.5)", width=10)
    )
    # endzone
    fig.add_shape(
        type="rect",
        x0=X_LEFT,
        y0=Y_MAX - 10,
        x1=X_RIGHT,
        y1=Y_MAX,
        line=dict(color="#39FF14", width=6),
        opacity=0.4,
    )
    # Add the path traces to the figure first to place them in the background
    # for trace in path_traces:
    #     fig.add_trace(trace)

    # set play speed
    frame_duration = 100
    for button in fig.layout.updatemenus[0].buttons:
        button["args"][1]["frame"]["duration"] = frame_duration
    # set aspect ratio
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    # background color
    fig.update_layout(paper_bgcolor="#333333", plot_bgcolor="#363636", font_color="white", font_size=14)
    # turn off axis
    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)
    # grid line thickness
    fig.update_yaxes(
        showgrid=True,
        gridwidth=3,
        gridcolor="rgba(237, 234, 222, 0.1)",
        linewidth=0,
        linecolor="rgba(0, 0, 0, 0.01)",
        mirror=True,
        showticklabels=False,
    )
    # set y axis range
    fig.update_yaxes(range=[mvmt_y_min, mvmt_y_max])
    # set yaxes ticks to 10 yards
    # fig.update_yaxes(tick0=0, dtick=10)
    # text size
    fig.update_layout(uniformtext_minsize=2, uniformtext_mode="hide")
    # hide legend
    fig.update_layout(showlegend=False)
    # text color of jersey numbers
    fig.update_traces(textfont=dict(family="Tahoma", size=12, color=mvmt_df["text_color"]))
    fig.update_traces(marker_line_width=0)

    # hide x and y labels
    fig.update_xaxes(title_text="")
    fig.update_yaxes(title_text="")

    # add hash marks, yard markers, etc
    for y_loc in range(Y_MIN + 10, Y_MAX - 10 + 1, 1):
        if y_loc % 10 == 0:
            ydln = y_loc - 10 if y_loc <= 60 else 110 - y_loc
            ydln_txt = str(ydln) if ydln != 0 else "E Z"
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=2), opacity=0.05
            )
            fig.add_annotation(
                x=X_LEFT + 4,
                y=y_loc,
                text=ydln_txt,
                showarrow=False,
                font=dict(color="white", size=60),
                textangle=90,
                opacity=0.05,
            )
            fig.add_annotation(
                x=X_RIGHT - 4,
                y=y_loc,
                text=ydln_txt,
                showarrow=False,
                font=dict(color="white", size=60),
                textangle=270,
                opacity=0.05,
            )
        elif y_loc % 5 == 0:
            fig.add_shape(
                type="line", x0=X_LEFT, y0=y_loc, x1=X_RIGHT, y1=y_loc, line=dict(color="white", width=1), opacity=0.05
            )
        else:
            fig.add_shape(
                type="rect",
                x0=X_MIDDLE - 9.5,
                y0=y_loc,
                x1=X_MIDDLE - 8.5,
                y1=y_loc,
                line=dict(color="white", width=3),
                opacity=0.05,
            )
            fig.add_shape(
                type="rect",
                x0=X_MIDDLE + 9.5,
                y0=y_loc,
                x1=X_MIDDLE + 8.5,
                y1=y_loc,
                line=dict(color="white", width=3),
                opacity=0.05,
            )

    fig.add_annotation(
        x=X_MIDDLE, y=Y_MAX - 5, text=off_club, showarrow=False, font=dict(color="white", size=90), opacity=0.2
    )
    fig.add_annotation(
        x=X_MIDDLE, y=Y_MIDDLE + 2, text="SŪMER", showarrow=False, font=dict(color="white", size=50), opacity=0.05
    )
    fig.add_annotation(
        x=X_MIDDLE, y=Y_MIDDLE - 2, text="SPORTS", showarrow=False, font=dict(color="white", size=50), opacity=0.05
    )

    # Add play description
    if play_description is not None:
        # make list of 100 character slices
        play_desc_list = [play_description[i : i + 100] for i in range(0, len(play_description), 100)]
        for i, play_desc_txt in enumerate(play_desc_list):
            text_y_loc = mvmt_y_min - 5 - 1 * i
            fig.add_annotation(
                x=X_MIDDLE,
                y=text_y_loc,
                text=play_desc_txt,
                showarrow=False,
                font=dict(color="white", size=14),
                opacity=0.8,
            )

    # set title
    # offense = play_info["offense"].values[0].upper()
    # defense = play_info["defense"].values[0].upper()
    # down = int(play_info["down"].values[0])
    # yards_to_go = int(play_info["yards_to_go"].values[0])
    # quarter = int(play_info["quarter"].values[0])
    # game_clock = play_info["game_clock"].values[0]
    fig.update_layout(
        title=f"{dataset_split} | {ballCarrierName} | {gameId} {playId} | {off_club} vs {def_club} | Down: {down} | YTG: {yards_to_go} | DTG: {distToGoal}",
        font_size=12,
        title_x=0.5,
        title_y=0.98,
    )

    return fig

In [8]:
# Random Sample from Test
gid, pid = (
    results_df.filter(pl.col("dataset_split").is_in(["test"])).select(["gameId", "playId"]).sample(1).to_numpy()[0]
)

#### Interesting Plays ####
# gid, pid = 2022100206, 988 # austin ekeler reverse TD, decent
# gid, pid = 2022102305, 380 # blocker causes prediction to jump
gid, pid = 2022103002, 3044 # Jet Sweep shows poor generalization from Zoo model, good
# gid, pid = 2022091109, 187 # James Rob. big run to left. Sumer model picks up earlier it is going to be a big run

print(f"gid, pid = {gid}, {pid}")
animate_play(tracking_df, play_df, results_df, gid, pid)

gid, pid = 2022103002, 3044
